# Demo for ACTS Assignments
#### Rodrigo Alejandro Chavez Mulsa


In [44]:
%load_ext autoreload
%autoreload 2
from modules.Classifier import Classifier
from pytorch_lightning import LightningModule

from pytorch_lightning import Trainer
from modules.AverageEmbeddings import AverageEmbeddings
from modules.Classifier import Classifier
import torch
from torchtext.legacy.datasets.nli import SNLI
from torchtext.legacy.data import Field
from torchtext.legacy import data
import torchtext
from torch.utils.data import DataLoader

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Functions

In [45]:
def get_checkpoint_path(model_name='awe'):
    """Get the path to the model checkpoints. Available: [awe, lstm, bilstm, bilstm-max]
    """
    return'trained_models/'+model_name+'/gold/'+model_name+'.ckpt'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [52]:
def get_prediction(_premise,_hypothesis,model):
    premise  = _premise.split(' ')
    hypothesis  = _hypothesis.split(' ')

    prem = TEXT.process([premise])
    hyp = TEXT.process([hypothesis])

    prem = [a.to(device) for a in prem] #Send to device
    hyp = [a.to(device) for a in hyp] 

    pred = model.demo_inference(prem,hyp)
    print("Premise:",_hypothesis,'\nHypothesis:',_hypothesis,"\nPredicted label -->",pred[1],'<-- with confidence:',pred[2])

## Load models

In [54]:
MODEL = 'bilstm'

In [46]:
glove = torchtext.vocab.GloVe(name='840B', dim=300)

In [47]:
TEXT = Field(lower=True, include_lengths=True, batch_first=True,tokenize='spacy',tokenizer_language="en_core_web_sm")
LABEL = Field(sequential=False)

train, dev, test = SNLI.splits(TEXT, LABEL, root= './data')
TEXT.build_vocab(train, vectors=glove)

In [55]:
model_name = MODEL#'lstm'
checkpath = get_checkpoint_path(model_name)
model = Classifier()
pretrained_model = model.load_from_checkpoint(checkpath,model_name=model_name,disable_nonlinear=True).to(device)
 

In [59]:
#Twice cells in case we want to input some sentences but want to keep the basic example.
premise = "Two woman are embracing while holding to go packages"
hypothesis = "Two woman are something packages"

In [57]:
premise = "Two woman are embracing while holding to go packages"
hypothesis = "Two woman are something packages"
get_prediction(premise,hypothesis,pretrained_model)

Premise: Two woman are something packages 
Hypothesis: Two woman are something packages 
Predicted label --> entailment <-- with confidence: 0.8692886829376221


## Scores 
In the following table we can see the accuracy scores for the NLI dev and test set along with the micro and macro scores of the transfer task which were measured by aggregating the accuracy scores of the following transfer tasks:
 
`transfer_tasks=['MR', 'CR', 'MPQA', 'SUBJ', 'SST2', 'TREC','MRPC', 'SICKEntailment']`

| Model      | NLI-dev | NLI-test | Transf-micro | Transf-macro |
|------------|---------|----------|--------------|--------------|
| AWE        | 0.6173  | 0.6283   | 82.573       | 79.129       |
| LSTM       | 0.791   | 0.7834   | 77.41        | 78.337       |
| BILSTM     | 0.7935  | 0.7948   | 83.36        | 82.185       |
| BILSTM-MAX | 0.834   | 0.8333   | 87.075       | 84.95        |

For the transfer tasks I used the suggested parameters to reproduce the results from the authors which are the following:    
`params_senteval = {'task_path': PATH_TO_DATA, 'usepytorch': True, 'kfold': 10}`    
`params_senteval['classifier'] = {'nhid': 0, 'optim': 'adam', 'batch_size': 64,'tenacity': 5, 'epoch_size': 4}`